In [26]:
import torch
from torch import nn
import torch.nn.functional as F

class CNN(nn.Module):
    """takes 4 consecutive frames, and compute the position and velocity of the thing"""
    def __init__(self, input_dim=6):
        super().__init__()
        self.down00 = nn.Conv2d(input_dim, 16, kernel_size = 3, padding = 1, stride = 2)
        self.down01 = nn.Conv2d(16, 16, kernel_size = 3, padding = 1, stride = 2)
            
        self.res00 = nn.Conv2d(16, 16, kernel_size = 3, padding = 1)
        self.res01 = nn.Conv2d(16, 16, kernel_size = 1)
        
        self.down10 = nn.Conv2d(16, 32, kernel_size = 3, padding = 1, stride = 2)
        self.down11 = nn.Conv2d(32, 32, kernel_size = 3, padding = 1, stride = 2)
        
        self.res10 = nn.Conv2d(32, 16, kernel_size = 3, padding = 1)
        self.res11 = nn.Conv2d(16, 32, kernel_size = 1)
        
        self.down2 = nn.Conv2d(32, 32, kernel_size = 3, padding = 1, stride = 2)
        
        self.res20 = nn.Conv2d(32, 16, kernel_size = 3, padding = 1)
        self.res21 = nn.Conv2d(16, 32, kernel_size = 1)
        
        self.down3 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1, stride = 2)
        
        self.res30 = nn.Conv2d(64, 32, kernel_size = 3, padding = 1)
        self.res31 = nn.Conv2d(32, 64, kernel_size = 1)
        
        self.down4 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1, stride = 2)
        
        # 7 7
        self.fc0 = nn.Linear(2*2*64, 256)
        self.fc1 = nn.Linear(256, 6)
        
    def forward(self, x):
        
        x = F.relu(self.down00(x))
        x = self.down01(x)
        # 64
        
        inter = F.relu(self.res00(x))

        x = x + self.res01(inter)
        
        x = F.relu(self.down10(x))
        x = self.down11(x)
        # 16
        
        inter = F.relu(self.res10(x))
        x  = x+ self.res11(inter)
        
        x = self.down2(x)
        # 8
        
        inter = F.relu(self.res20(x))
        x = x + self.res21(inter)
        
        x = self.down3(x)
        
        # 4
        
        inter = F.relu(self.res30(x))
        x = x + self.res31(inter)
        
        x = self.down4(x)
        # 2

        x = F.relu(self.fc0(x.view(-1, 2*2*64)))
        x = self.fc1(x)
        return x

In [27]:
net = CNN()

In [28]:
net(torch.zeros((3, 6, 256, 256)))

tensor([[ 0.0656, -0.1008,  0.0906, -0.0195,  0.0335,  0.0402],
        [ 0.0656, -0.1008,  0.0906, -0.0195,  0.0335,  0.0402],
        [ 0.0656, -0.1008,  0.0906, -0.0195,  0.0335,  0.0402]],
       grad_fn=<AddmmBackward>)